In [1]:
import numpy as np
import pandas as pd
import bloscpack as bp

import imgaug as ia
import imgaug.augmenters as iaa

In [2]:
import fastai
from fastai.vision import *

from optim import Over9000

In [3]:
from torch.utils.data.dataloader import DataLoader
from data import DS_TRN

In [4]:
from model import *
from model_utils import *

---

In [5]:
SEED = 20190819

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

---
### data

In [6]:
trn_imgs = bp.unpack_ndarray_from_file('../features/train_images.bloscpack')
# see preprocess notebook, it's shown that the train.csv data sequence is the same and can just load like this.
trn_lbls = pd.read_csv('../input/train.csv').iloc[:, 1:4].values

In [7]:
augs = iaa.SomeOf(
    (0, 3),
    [
        iaa.OneOf(
            [
                iaa.Affine(scale={"x": (0.8, 1.1), "y": (0.8, 1.1)}, rotate=(-15, 15), shear=(-15, 15)),
                iaa.PiecewiseAffine(scale=(0.02, 0.03)),
                iaa.PerspectiveTransform(scale=.09, keep_size=True),
            ]
        ),
        iaa.GaussianBlur(sigma=(0.5, 1.)),
#         iaa.OneOf(
#             [
#                 iaa.GaussianBlur(sigma=(0.5, 1.)),
#                 iaa.MotionBlur(k=(7, 13), angle=[-45, 45]),
#                 iaa.MedianBlur(k=(3, 5)),
#             ]
#         ),
        iaa.OneOf(
            [
                iaa.DirectedEdgeDetect(alpha=(.6, .8), direction=(0.0, 1.0)),
                iaa.Emboss(alpha=(.5, 1.), strength=(.1, 4)),
            ]
        ),
#         iaa.CoarseDropout((0.05, 0.15), size_percent=(0.15, 0.35)),
#         iaa.Fog(),
    ],
    random_order=True
)

In [ ]:
# stratified folds

In [8]:
training_set = DS_TRN(trn_imgs[:180000, np.newaxis], trn_lbls[:180000], transform=augs)
validation_set = DS_TRN(trn_imgs[180000:, np.newaxis], trn_lbls[180000:])

training_loader = DataLoader(training_set, batch_size=64, shuffle=True, num_workers=6)
validation_loader = DataLoader(validation_set, batch_size=64, shuffle=False, num_workers=6)

data_bunch = DataBunch(train_dl=training_loader, valid_dl=validation_loader)

---
### model

In [10]:
device = 'cuda:0'
n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant

In [11]:
predictor = PretrainedCNN(out_dim=n_total)
classifier = BengaliClassifier(predictor)#.to(device)

In [12]:
learn = Learner(
    data_bunch,
    classifier,
    loss_func=Loss_combine(),
    opt_func=Over9000,
    metrics=[Metric_grapheme(), Metric_vowel(), Metric_consonant(), Metric_tot()]
)

logger = CSVLogger(learn, 'Seresnext_DensenetStarterSetup_Myaug_Mixup')

learn.clip_grad = 1.0
learn.split([classifier.predictor.lin_layers])
# learn.split([classifier.head1])
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(
    32,
    max_lr=slice(0.2e-2,1e-2),
    wd=[1e-3, 0.1e-1],
    pct_start=0.0,
    div_factor=100,
    callbacks=[logger, SaveModelCallback(learn, monitor='metric_tot', mode='max', name='Seresnext_DensenetStarterSetup_Myaug_Mixup'), MixUpCallback(learn)] # 
)

epoch,train_loss,valid_loss,metric_idx,metric_idx,metric_idx,metric_tot,time
0,1.354018,0.406156,0.842983,0.924481,0.942021,0.888117,16:04
1,1.185271,0.293037,0.901694,0.956087,0.938648,0.924531,16:03
2,1.080569,0.208451,0.931433,0.976174,0.965027,0.951017,16:05
3,0.990788,0.192998,0.939203,0.978271,0.961679,0.954589,16:05
4,0.938661,0.165359,0.945129,0.977432,0.972682,0.960093,16:06
5,0.896341,0.151977,0.949640,0.977125,0.978090,0.963624,16:04
6,0.821919,0.155075,0.953000,0.979472,0.973237,0.964677,16:06
7,0.806160,0.146749,0.951435,0.979082,0.977711,0.964916,16:06
8,0.765637,0.144403,0.951363,0.983733,0.971789,0.964562,16:05
9,0.730734,0.142898,0.953666,0.980960,0.981062,0.967339,16:05
